# Section 2

## This is to ensure you have the proper tools installed

In [ ]:
%pip install pandas
%pip install mysql-connector-python
%pip install SQLAlchemy
%pip install kagglehub

## This checks if the necessary files are accessible and available

If the csv file can not be found download here and make sure it is in the correct directory:

https://drive.google.com/file/d/1B0P20CU9JP1I-jj0sgnq4QRvrtRLgUxF/view?usp=drive_link

In [ ]:
from pathlib import Path 
import os

csv_file = "IMDB Movie Big Dataset.csv"
current_dir = Path(os.getcwd())

if (current_dir / csv_file).is_file():
    csv_file = (current_dir / csv_file)
    print("CSV present")
else:
    import kagglehub
    cached_path = Path(kagglehub.dataset_download("shubhamchandra235/imdb-and-tmdb-movie-metadata-big-dataset-1m"))
    if cached_path.is_file():
        csv_file = cached_path
    elif cached_path.is_dir():
        csv_file = list(cached_path.glob("*.csv"))[0]
        target_dir = str(current_dir) + "/IMDB Movie Big Dataset.csv"
        os.rename(csv_file, target_dir)
        csv_file = target_dir
        print("Successfully downloaded and moved file")

In [ ]:
import ast
import sqlite3
import pandas as pd

# Turns genres_list from string representation to array 
df = pd.read_csv(csv_file)
df['genres_list'] = df['genres_list'].apply(ast.literal_eval)

# Renames variables to better names
df.rename(columns={
    'Director':'director',
    'genres_list':'genres'
}, inplace=True)

conn = sqlite3.connect("imdb.sqlite")
cursor = conn.cursor()

In [ ]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS movies (
    movie_id INTEGER PRIMARY KEY,
    title TEXT,
    revenue INTEGER,
    budget INTEGER,
    vote_count INTEGER,
    vote_average REAL,
    runtime INTEGER,
    director TEXT
    )
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS genres (
    genre_id INTEGER PRIMARY KEY,
    name TEXT UNIQUE
    )
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS movie_genres (
    movie_id INTEGER,
    genre_id INTEGER,
    PRIMARY KEY(movie_id, genre_id),
    FOREIGN KEY(movie_id) REFERENCES movies(movie_id),
    FOREIGN KEY(genre_id) REFERENCES genres(genre_id)
    )
""")

conn.commit()

In [ ]:
movies = df[['title', 'revenue', 'budget', 'vote_count', 'vote_average', 'runtime', 'director']].drop_duplicates()
movies.to_sql('movies', conn, if_exists='replace', index=True, index_label='movie_id')

genre_list = set(g for sublist in df['genres'] for g in sublist)
genres = pd.DataFrame({'name': list(genre_list)})
genres.to_sql('genres', conn, if_exists='replace', index=True, index_label='genre_id')

In [ ]:
# Sets up the many to many relationship
genres_df = pd.read_sql("SELECT * FROM genres", conn)
genre_map = dict(zip(genres_df['name'], genres_df['genre_id']))

movie_genres = []
for _, row in df.iterrows():
    movie_id = row['movie_id']
    for g in row['genres']:
        movie_genres.append({'movie_id': movie_id, 'genre_id': genre_map[g]})

movie_genres_df = pd.DataFrame(movie_genres)
movie_genres_df.to_sql('movie_genres', conn, if_exists='replace', index=False)

### TEMP to show how to access the SQLite database

In [ ]:
# TEMP to show how to access movies
movies_df = pd.read_sql("SELECT * FROM movies", conn)
movies_df.head(50)

In [ ]:
# TEMP TO show how to access genres
genres_df = pd.read_sql("SELECT * FROM genres", conn)
genres_df.head(30)

In [ ]:
movie_genres_df = pd.read_sql("""
    SELECT m.title, g.name AS genre
    FROM movie_genres mg
    JOIN movies m ON mg.movie_id = m.movie_id
    JOIN genres g ON mg.genre_id = g.genre_id
    """, conn)

movie_genres_df.head(15)

## Naïve Implementetation